In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, label_binarize
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc

from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

# Veri yükleme
data = pd.read_excel('paper.xlsx')

# GPA'yı sınıflara ayırma
def gpa_to_label(gpa):
    if gpa >= 3.0:
        return 'High'
    elif gpa >= 2.0:
        return 'Medium'
    else:
        return 'Low'

data['GPA_Class'] = data['GPA'].apply(gpa_to_label)

# Eksik verileri temizleme
data = data.dropna()

# Sayısal sütunlar (varsa)
numeric_columns = ['Time_Friends', 'Time_SocicalMedia', 'Time_Studying']
numeric_columns = [col for col in numeric_columns if col in data.columns]

# Özellikler ve hedef değişken
X = data.drop(['GPA', 'GPA_Class'], axis=1)

# Sayısal sütunları ölçeklendirme
scaler = StandardScaler()
X[numeric_columns] = scaler.fit_transform(X[numeric_columns])

# Hedef değişkeni encode et
le = LabelEncoder()
y_encoded = le.fit_transform(data['GPA_Class'])

# SMOTE ile dengeleme
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y_encoded)

# One-hot encode (Keras için)
y_categorical = to_categorical(y_resampled)

# Train-test böl
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_categorical, test_size=0.2, random_state=42)

# === ROC için sınıfları binarize et ===
y_train_bin = label_binarize(y_train, classes=['Low', 'Medium', 'High'])
y_test_bin = label_binarize(y_test, classes=['Low', 'Medium', 'High'])

# Model oluştur
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(y_categorical.shape[1], activation='softmax')
])

# Derle
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Eğit
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=2)

# Değerlendir
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test doğruluğu: {accuracy:.4f}")

# Tahmin
y_pred_prob = model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)
y_true = np.argmax(y_test, axis=1)

# Sınıflandırma Raporu
print("Sınıflandırma Raporu:")
print(classification_report(y_true, y_pred, target_names=le.classes_))
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

# === ROC Eğrisi ===
# Her sınıf için ROC eğrisi çizimi (One-vs-Rest)
fpr, tpr, roc_auc = {}, {}, {}
n_classes = y_test.shape[1]
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_pred_prob[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# ROC eğrisini çiz
plt.figure(figsize=(8, 6))
colors = ['blue', 'orange', 'green']
labels = le.classes_
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i], color=colors[i], lw=2, label=f'{labels[i]} (AUC = {roc_auc[i]:.2f})')

plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Eğrisi - Derin Öğrenme Modeli")
plt.legend(loc="lower right")
plt.grid()
plt.tight_layout()
plt.show()
